### Download the Dataset

In [1]:
!wget -O "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/BanglaWriting.zip" \
      https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/r43wkvdk4w-1.zip

--2022-01-09 08:29:44--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/r43wkvdk4w-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.57.72
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.57.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 268633599 (256M) [application/octet-stream]
Saving to: ‘/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/BanglaWriting.zip’

100%[======================================>] 268,633,599 23.5MB/s   in 12s    

2022-01-09 08:29:57 (21.4 MB/s) - ‘/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/BanglaWriting.zip’ saved [268633599/268633599]



### Unzip the Dataset

In [2]:
import os

!unzip -qq "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/BanglaWriting.zip" -d "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/"
!unzip -qq "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/converted.zip" -d "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/BanglaWriting/"
!unzip -qq "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/raw.zip" -d "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/BanglaWriting/"

os.remove("/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/converted.zip")
os.remove("/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/raw.zip")

### Import Dependencies

In [16]:
import os
import glob
import json
import random
from tqdm import tqdm

from PIL import Image
import cv2

DATA_DIR = "/home/ec2-user/word_level_ocr/pritom/datasets/handwriting/"

### Extract Words

In [4]:
WIDTH = 128
HEIGHT = 512
DIM = (WIDTH, HEIGHT)

dataset_path = os.path.join(DATA_DIR, "BanglaWriting")
extracted_path = os.path.join(DATA_DIR, "BanglaWriting_words")

try:
    # Create  Directory  MyDirectory 
    os.mkdir(extracted_path)
except FileExistsError:
    ##print if directory already exists
    print("Directory already exists.")

for sub_dir in os.listdir(dataset_path):

    dataset_sub_dir = os.path.join(dataset_path, sub_dir)

    try:
        # Create  Directory  MyDirectory 
        os.mkdir(os.path.join(extracted_path, sub_dir))
    except FileExistsError:
        ##print if directory already exists
        print("Directory already exists.")

    word_counter = 0

    for filename in tqdm(os.listdir(dataset_sub_dir)):
        
        if filename.endswith(".json"):

            json_file = open(os.path.join(dataset_sub_dir, filename), encoding='utf-8')
            page = json.load(json_file)

            # take the image file name from the json file
            # image_path = page["imagePath"]

            # if not image_path[0].isdigit():
            #     image_path = image_path.rsplit("\\", 1)[1]

            image_path = filename.replace(".json", ".jpg")
            image = cv2.imread(os.path.join(dataset_sub_dir, image_path))
            image_path = image_path.replace(".jpg", "")

            # faulty_pages = ["256_14_1/", "25_22_0/"]
            # if image_path in faulty_pages:okb ===============ntinue

            # print("Extracting from " + image_path + " | words in page: " + str(len(page["shapes"])))

            for i in range(len(page["shapes"])):

                word_counter += 1

                label = page["shapes"][i]["label"] # label of the word
                # pixel coordinates of the bounding box
                xmin, ymin = page["shapes"][i]["points"][0]
                xmax, ymax = page["shapes"][i]["points"][1]

                word = image[int(ymin):int(ymax), int(xmin):int(xmax)]

                # cv2.imshow("example", word)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
                
                if word.shape[0] == 0 or word.shape[1] == 0:
                    continue
                
                word_resized = cv2.resize(word, DIM)
                file_name = "{}.jpg".format(word_counter)
                cv2.imwrite(os.path.join(extracted_path, sub_dir, file_name), word_resized)
                
                with open(os.path.join(extracted_path, "{}_labels.txt".format(sub_dir)), "a", encoding="utf8") as f:
                    f.write("{} {}\n".format(file_name, label))
                    


100%|█████████████████████████████████████████████████████████████| 520/520 [00:48<00:00, 10.83it/s]


### Split Train and Validation Files

In [47]:
extracted_path = os.path.join(DATA_DIR, "BanglaWriting_words")

labels_files = glob.glob(extracted_path + "/*labels.txt")

for label_file in labels_files:
    current_dir = label_file.rsplit('/', 1)[1].split('_', 1)[0]

    with open(label_file) as f:
        lines = f.readlines()

    random.shuffle(lines)

    train_split = int(0.90 * len(lines))
    valid_split = train_split + int(0.05 * len(lines))

    train_data = lines[:train_split]
    valid_data = lines[train_split:valid_split]
    test_data = lines[valid_split:]

    with open(os.path.join(extracted_path, "{}_labels_train.txt".format(current_dir)), 'w') as fout:
        for line in train_data:
            fout.write(line)

    with open(os.path.join(extracted_path, "{}_labels_valid.txt".format(current_dir)), 'w') as fout:
        for line in valid_data:
            fout.write(line)

    with open(os.path.join(extracted_path, "{}_labels_test.txt".format(current_dir)), 'w') as fout:
        for line in test_data:
            fout.write(line)